# Rebalance instances

### 1) Overall configurations 

In [19]:
import os
import os.path
import pandas as pd
from pprint import pprint

# Root
root_path = os.getcwd().replace("\\","/")

# Instance settings
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/rebalancing/instance_settings_test_rebalancing.json"
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/allow_hiring.json"
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/rebalancing/no_rebalancing_fast.json"


dict_sl_status = {"MET":"Met", "UNMET":"Unmet"}

dict_sq_class = {"A":"Business", "B":"Standard", "C":"Low-cost"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["Business", "Standard", "Low-cost"], ordered=True)



### 2) Loading the instance data

In [20]:
import json

def load_json(path):
    """Read json file and return dictionary"""

    # Add .json to the end of file if needed
    if path.find(".json") < 0:
        path = path + ".json"

    # Read JSON file
    with open(path) as data_file:
        data_loaded = json.load(data_file)

    return data_loaded

instances_dic = load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]
request_log_folder = result_folder + "/request_track"

# Folder where instances are located
instances_folder = instances_dic["instances_folder"]

# File name aggregated data
instance_name = instances_dic["instance_name"]


print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'generate_test_cases': True,
 'instance_description': 'Baseline scenario, fixed fleet and no rebalancing '
                         '10min',
 'instance_name': 'fixedFleetNoRebalFast',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/fixed_fleet_no_rebalancing_fast/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_duration',
            'CS': 'customer_segmentation',
            'CT': 'clear_target_list_every_round',
            'ID': 'instance_description',
            'IF': 'initial_fleet',
            'IN': 'instance_name',
            'MC': 'max_capacity',
            'MO': 'allow_many_to_one',
            'MR': 'max_requests',
            'RE': 'rebalance',
            'RT': 'reinsert_targets',
            'SD': 'allow_service_deterioration',
            'SR': 'service_rate',
            'ST': 'simulation_time',
            'UR': 'allow_urgent_relocation'

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output = 
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [21]:
def get_instance_settings(file_name):
    """ Read file name and return instance settings.
    E.g.:
     Input = IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR
     Output = {'allow_many_to_one': True,
                'allow_service_deterioration': True,
                'allow_urgent_relocation': True,
                'allow_vehicle_hiring': True,
                'batch_duration': '30',
                'clear_target_list_every_round': True,
                'contract_duration': '3600',
                'customer_segmentation': 'AA',
                'initial_fleet': '1000',
                'max_capacity': '06',
                'max_requests': '1000',
                'reinsert_targets': True,
                'time_horizon': '86400'}
    """
    label_setting_dic = instances_dic["labels"]
    
    print(file_name)
        
    # E.g., ['BA-30', 'TH-86400', 'MR-1000', 'IF-1000', 'MC-06', 'CD-3600-SR-S1', 'CS-AA', 'SD', 'VH', 'MO', 'RT', 'CT', 'UR']
    file_instances = file_name.split("_")

    instance_settings = dict()

    for e in file_instances:
        
        if e in label_setting_dic.keys():
            # E.g., e =  SD
            k = label_setting_dic[e]
            # E.g., k = allow_service_deterioration
            instance_settings[k] = True
            
        else:
            # E.g., lv = ["BA", "30"]
            lv  = e.split('-')
            # E.g., e2 = BA
            e2 = lv[0]
            # E.g., k = batch_duration
            k2 = label_setting_dic[e2]
            
            if len(lv) > 1:
                 # E.g., v = '30'
                v = lv[1]
                instance_settings[k2] = v
            else:
                # label is not in instance name = False
                instance_settings[k] = False

    return instance_settings

### Aggregate results (folder round_track)

Instance fields in `round_track` folder. Every line is a snapshot of a simulation round of 30 seconds (first column is round `timestamp`):

#### Request status per round
* `waiting`
* `finished`
* `denied`
* `n_requests`

#### Freelance vehicles per round
* `hired_vehicles`
* `deactivated_vehicles`

#### Vehicle status per round
* `active_vehicles`
* `enroute_count`
* `parked_vehicles`
* `origin_vehicles`
* `rebalancing`
* `stopped_rebalancing`
* `idle`
* `picking_up`
* `O1,O2,O3,O4`
* `V1,V2,V3,V4`
* `distance_traveled_cruising`
* `distance_traveled_loaded`
* `distance_traveled_rebalancing`
* `run_time`

#### Vehicle status per round (seats)
* `seat_count`
* `picking_up_seats`
* `rebalancing_seats`
* `empty_seats`
* `total_capacity`

#### Service quality
* `pk_delay`
* `total_delay`
* `A_pk,A_dp,A_count,A_unmet_slevels`
* `B_pk,B_dp,B_count,B_unmet_slevels`
* `C_pk,C_dp,C_count,C_unmet_slevels`

In [22]:
import pandas as pd

def get_results_dic(path_experiment, name_experiment):
    
    # Initial capacity to subtract from total number of seats
    capacity_vehicle_initial_fleet = 4
    n_initial_fleet = 1000
    capacity_initial_fleet = capacity_vehicle_initial_fleet * n_initial_fleet

    # Load results
    experiment_file = "{}round_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
   

    df = pd.read_csv(experiment_file, index_col="timestamp",  parse_dates = True)

    # Number of requests
    total_requests = df["n_requests"].sum()
    serviced = df["finished"][-1]
    denied = df["denied"][-1]
    
    # Service quality
    sq_classes = ["A", "B", "C"]
    sq_settings = ["pk", "dp", "count", "unmet_slevels"]
    # e.g., ['A_pk', 'A_dp', 'A_count', 'A_unmet_slevels', 'B_pk', 'B_dp', 'B_count', 'B_unmet_slevels', 'C_pk', 'C_dp', 'C_count', 'C_unmet_slevels']
    sq_user_labels = ["{}_{}".format(sq_class, sq_setting) for sq_class in sq_classes for sq_setting in sq_settings]
    sq_user_dic = {l:df[l][-1] for l in sq_user_labels}
    
    # Distance traveled
    distance_cruising = df["distance_traveled_cruising"][-1]
    distance_loaded = df["distance_traveled_loaded"][-1]
    distance_rebalancing = df["distance_traveled_rebalancing"][-1]
    distance_total = distance_cruising + distance_loaded + distance_rebalancing

    # Pickup
    avg_pk_delay = df["pk_delay"].mean()
    avg_ride_delay = df["total_delay"].mean()

    # Separate occupancy labels (e.g., O1, O2, O3, etc.)
    occupancy_labels = [col for col in list(df) if 'O' in col]
    status_labels = [ "idle", "picking_up"] + occupancy_labels
    
    
    # Get fleet makeup
    #fleet_makeup_labels = [col for col in list(df) if 'V' in col]
    #fleet_makeup = {mk:df[mk][-1] for mk in fleet_makeup_labels}
    #total_seats = {"seats_" + k:int(k[1:]) * v for k,v in fleet_makeup.items()}
    #total_seats["total_seats"] = sum(total_seats.values())
    
    # Fleet statistics
    avg_seats = df["total_capacity"].mean() - capacity_initial_fleet
    median_seats = df["total_capacity"].median() - capacity_initial_fleet
    max_seats = df["total_capacity"].max() - capacity_initial_fleet
    id_max_seats = df["total_capacity"].idxmax()
    max_hired = df["hired_vehicles"].max() - capacity_initial_fleet
    median_hired = df["hired_vehicles"].median() - capacity_initial_fleet
    mean_hired = df["hired_vehicles"].mean() - capacity_initial_fleet
    mean_active = df["active_vehicles"].mean() - capacity_initial_fleet
    occupancy = (df["seat_count"]/df['total_capacity']).mean()
    
    # How many vehicles are active (i.e., servicing customers)?
    total_occupied = (df["O1"] + df["O2"] + df["O3"] + df["O4"])
    o1 = (df["O1"]/total_occupied).mean()
    o2 = (df["O2"]/total_occupied).mean()
    o3 = (df["O3"]/total_occupied).mean()
    o4 = (df["O4"]/total_occupied).mean()
    
    # How many vehicle of each capacity hired?
    total_hired = (df["V1"] + df["V2"] + df["V3"] + (df["V4"])-1000)
    v1 = (df["V1"]/total_hired).mean()
    v2 = (df["V2"]/total_hired).mean()
    v3 = (df["V3"]/total_hired).mean()
    v4 = ((df["V4"]-1000)/total_hired).mean()
          
    idle_mean = df['idle'].mean()
    idle_pickingup = df['picking_up'].mean()
    
    servicing_seats = (df['seat_count']/df["total_capacity"]).mean()
    picking_up_seats = (df['picking_up_seats']/df["total_capacity"]).mean()
    rebalancing_seats = (df['rebalancing_seats']/df["total_capacity"]).mean()
    parked_seats = (df['empty_seats']/df["total_capacity"]).mean()

    # Filter occupancy columns
    df_occupancy = df[occupancy_labels]
    
    # Build fleet status
    df_status = pd.DataFrame(df_occupancy)
    df_status["picking_up"] = df["picking_up"]
    df_status["idle"] = df["idle"]
    
    # Smooth values
    df_occupancy = df_occupancy.rolling(window=24).mean()

    # Runtime
    total_runtime = df["run_time"].sum()
    
    avg_runtime = df["run_time"].mean()
    
    # Dictionary of agreggate data
    dic_agreggate_data = {
                        "serviced_seats": "{:.2%}".format(servicing_seats),
                        "picking_up_seats": "{:.2%}".format(picking_up_seats),
                        "rebalancing_seats": "{:.2%}".format(rebalancing_seats),
                        "parked_seats": "{:.2%}".format(parked_seats),
                        "serviced": "{:.2%}".format(serviced/total_requests),
                          "denied": "{:.2%}".format(denied/total_requests),
                          'max_hired': max_hired,
                          'occupancy':"{:.2%}".format(occupancy),
                          'o1':"{:.2%}".format(o1),
                          'o2':"{:.2%}".format(o2),
                          'o3':"{:.2%}".format(o3),
                          'o4':"{:.2%}".format(o4),
                          'v1':"{:.2%}".format(v1),
                          'v2':"{:.2%}".format(v2),
                          'v3':"{:.2%}".format(v3),
                          'v4':"{:.2%}".format(v4),
                          'mean_hired': "{:.2f}".format(mean_hired),
                          'median_hired': median_hired,
                          'avg_seats': "{:.2f}".format(avg_seats),
                          'max_seats': max_seats,
                          'id_max_seats': id_max_seats,
                          'median_seats': median_seats,
                          'mean_active': mean_active,
                          "total_requests": total_requests,
                          "avg_pk_delay": "{:.2f}".format(avg_pk_delay),
                          "avg_ride_delay": "{:.2f}".format(avg_ride_delay),
                          "total_runtime": "{:.2f}".format(total_runtime/1000/60),
                          "avg_runtime": "{:.2f}".format(avg_runtime/1000),
                          "distance_cruising": "{:.2%}".format(distance_cruising/distance_total),
                          "distance_loaded": "{:.2%}".format(distance_loaded/distance_total),
                          "distance_rebalancing": "{:.2%}".format(distance_rebalancing/distance_total),
                          "distance_total": distance_total//1000}
    
    # All data
    ##dic_ag = {**dic_agreggate_data, **fleet_makeup}
    ##dic_ag = {**dic_ag, **total_seats}
    #dic_ag = {**dic_ag, **sq_user_dic}
    return dic_agreggate_data
    #return dic_ag

### Aggregate results (folder request_track)

* `earliest`
* `id` = 1, 2, 3, ..., #USERS
* `class` = A, B, C
* `pk_delay`
* `ride_delay`
* `pk_time`
* `dp_time`
* `id_from` = Network id
* `id_to` = Network id
* `dist` = trip(id_from, id_to) in seconds
* `service` = {FLEET, FREELANCE}
* `service_level` = {MET, UNMET}

import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    aggfunc = {"pk_delay" : ['mean', 'count', 'max']}
    
    dfp = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["pk_delay"])

    return dfp

In [23]:
import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    service_quality_dic = dict()
 
    for service_level in ["MET", "UNMET"]:
        total = df[(df['service_level'] == service_level)]["pk_delay"].count()
        for sq_class in ['A', 'B', 'C']:
            filter_sq_sl = (df['class'] == sq_class) & (df['service_level'] == service_level)
            #service_quality_dic["{}_{}".format(service_level, sq_class)] = df.loc[filter_sq_sl]["pk_delay"].count()
            if service_level == 'MET':
                service_quality_dic["{}_{}".format(service_level, sq_class)] = df.loc[filter_sq_sl]["pk_delay"].mean()
            else:
                service_quality_dic["{}_{}".format(service_level, sq_class)] = df.loc[filter_sq_sl]["pk_delay"].count()
                
        service_quality_dic["{}_TOTAL".format(service_level)] = total
    
    return service_quality_dic

### Processing single instance

In [24]:
import pandas as pd

def get_aggregate_request_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
   
    df = pd.read_csv(experiment_file, index_col = "earliest",  parse_dates = True)

    return df

# Rebalance and no rebalance aggregate data

In [25]:
# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)

instance_file_names = [i for i in instance_file_names if instances_dic['instance_name'] in i]

# Instance Info (key = instance name)
dic_all = dict()
print("Reading files in folder:", request_log_folder)

count = 1
for file_name in instance_file_names:
    
    
        
    instance, extension = file_name.split(".")
    
    # Instance settings
    instance_settings_dic = get_instance_settings(instance)
    if instance_settings_dic['instance_name'] != 'WEEKDENY' or instance_settings_dic['customer_segmentation'] in ['A', 'B', 'C']:
        continue
        
    print("  - Processing", instance)
        
    count = count + 1
    round_track_agg_dic = get_results_dic(instances_folder, instance)
    request_track_agg_dic = get_request_track_dic(instances_folder, instance)
    
    # Get aggregated results
    dic_all[instance] = {**instance_settings_dic, **round_track_agg_dic, **request_track_agg_dic}
    

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/fixed_fleet_no_rebalancing_fast//request_track
IN-fixedFleetNoRebalFast_BA-030_ST-00600_MR-1000_IF-01000_MC-06_CD-600_SR-S1_CS-CC


In [26]:
df_no_hiring = pd.DataFrame.from_dict(dic_all, orient='index')
df_no_hiring.to_csv("rebal_and_no_rebal.csv")
df_no_hiring

""


In [27]:
b = df_no_hiring[df_no_hiring['instance_name']=='WEEKDENY']

key_cs = 'customer_segmentation' # (A, AA, BB, etc.)
key_reb = 'rebalance'

b = b[[
    'rebalance',
    'customer_segmentation',
    'mean_active',
    'occupancy',
    'distance_cruising',
    'distance_loaded',
    'distance_rebalancing',
    'distance_total'
]+["{}_{}".format(service_level, sq_class) for sq_class in ['A', 'B', 'C'] for service_level in ["MET", "UNMET"]]+ ['MET_TOTAL', 'UNMET_TOTAL']]

# Filtering data
b = b[b[key_cs].isin(["AA", "BB", "CC"])]

# convert just columns "a" and "b"
#a[['contract_duration', 'median_seats', 'distance_total']] = a[['contract_duration', 'median_seats', 'distance_total']].apply(pd.to_numeric)

b = b.astype({'distance_total':int})
# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3"], ordered=True)


# Renaming data and applying aliases
b[key_cs] = b[key_cs].map(lambda e:dict_segmentation[e])
b[key_cs] = b[key_cs].astype(category_segmentation)

b[key_reb] = b[key_reb].map(lambda e:( "YES" if e == True else "NO"))

b = b.sort_values(by=[key_reb, key_cs])

b.rename(columns={'customer_segmentation': 'Customer segmentation',
                    'rebalance': 'Rebalance',
                    'mean_active': '#Active vehicles/Round',
                    'occupancy': 'Occupancy/Round',
                    'distance_cruising': 'Cruising',
                    'distance_rebalancing': 'Rebalancing',
                    'distance_loaded': 'Servicing',
                    'distance_total': 'Total (Km)'}, inplace=True)

b = b.set_index([ 'Rebalance' , 'Customer segmentation'])

b['#Active vehicles/Round'] = b['#Active vehicles/Round'].apply(lambda x:"{:.2f}".format(x))

b = b[['#Active vehicles/Round', 'Occupancy/Round', 'MET_A', 'MET_B', 'MET_C', 'MET_TOTAL', 'UNMET_A', 'UNMET_B', 'UNMET_C', 'UNMET_TOTAL',  'Cruising', 'Servicing', 'Rebalancing', 'Total (Km)'] ]
b["Service rate"] = (b['MET_TOTAL']/(b['MET_TOTAL'] + b['UNMET_TOTAL'])).apply(lambda x: "{:.2%}".format(x))

KeyError: 'instance_name'

### Service info (Rebalancing X No Rebalancing)

In [ ]:
#df_service = b[["Service rate", 'MET_A', 'MET_B', 'MET_C', 'MET_TOTAL', 'UNMET_A', 'UNMET_B', 'UNMET_C', 'UNMET_TOTAL']]

df_service = b[["Service rate", 'MET_A', 'MET_B', 'MET_C', 'UNMET_A', 'UNMET_B', 'UNMET_C', 'UNMET_TOTAL']]


df_service['MET_A'] =df_service['MET_A'].apply(lambda e: ('{:02}:{:02}'.format(int(e)//60, int(e)%60) if e != 0 else '-'))
df_service['MET_B'] = df_service['MET_B'].apply(lambda e: ('{:02}:{:02}'.format(int(e)//60, int(e)%60) if e != 0 else '-'))
df_service['MET_C'] = df_service['MET_C'].apply(lambda e: ('{:02}:{:02}'.format(int(e)//60, int(e)%60) if e != 0 else '-'))



print(df_service.to_latex(multicolumn=True, multirow=True, column_format='ccccccllll'))
df_service

### Fleet operation info (Rebalancing X No Rebalancing)

In [ ]:
df_fleet = b[['Occupancy/Round', '#Active vehicles/Round', 'Cruising', 'Servicing', 'Rebalancing', 'Total (Km)']]
df_fleet['#Active vehicles/Round'] = (df_fleet['#Active vehicles/Round'].astype(float)/1000).apply(lambda x: "{:.2%}".format(x))
print(df_fleet.to_latex(multicolumn=True, multirow=True, column_format='ccccccc'))
df_fleet

# Hiring aggregate data - Fleet stats

### Processing all instances in folder

In [ ]:
# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)


# Instance Info (key = instance name)

print("Reading files in folder:", request_log_folder)
dic_all_round_track = dict()

for file_name in instance_file_names:
    
    instance, extension = file_name.split(".")
        
    # Instance settings
    instance_settings_dic = get_instance_settings(instance)
    
    if instance_settings_dic['instance_name'] == "HIRINGWEEKMAXWAITINGREB2" or 'rebalance' not in instance_settings_dic.keys():
        continue
    if instance_settings_dic['customer_segmentation'] in ['A', 'B', 'C']:
        continue
  
    #if instance_settings_dic['instance_name'] != 'HIRINGWEEKMAXWAITING':
    #    continue
        
    print("  - Processing", instance)
    
    round_track_agg_dic = get_results_dic(instances_folder, instance)
    
    # Get aggregated results
    dic_all_round_track[instance] = {**instance_settings_dic, **round_track_agg_dic}

#print("Instance settings:")
#pprint(dic_instance_settings)

In [ ]:
df = pd.DataFrame.from_dict(dic_all_round_track, orient='index')
df.to_csv("data1.csv")

# Getting latex table
# print(a.to_latex(multicolumn=True, multirow=True))

df

df=df.fillna(-1)
df

In [ ]:

a = df#[df['instance_name']=='HIRINGWEEKMAXWAITING']

a = a[[
    'service_rate',
    'customer_segmentation',
    'contract_duration',
    'occupancy',
    'o1',
    'o2',
    'o3',
    'o4',
    'v1',
    'v2',
    'v3',
    'v4',
    'avg_seats',
    'id_max_seats',
    'max_seats',
    'median_seats',
    'distance_cruising',
    'distance_loaded',
    'distance_rebalancing',
    'distance_total',
    "serviced_seats",
    "picking_up_seats",
    "rebalancing_seats",
    "parked_seats",
    "avg_runtime"
    
]]

key_sr = 'service_rate' # (S1, S2, S3)
key_cs = 'customer_segmentation' # (A, AA, BB, etc.)
key_cd = 'contract_duration' #(0, 3600, 18000)


# Filtering data
a = a[a[key_cs].isin(["AA", "BB", "CC"])]

# convert just columns "a" and "b"
#a[['contract_duration', 'median_seats', 'distance_total']] = a[['contract_duration', 'median_seats', 'distance_total']].apply(pd.to_numeric)

a = a.astype({'contract_duration':int, 'median_seats':int, 'distance_total':int, 'max_seats': int})
# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_contract_duration = {3600:"1h", 0:"Single-ride", 10800:"3h", -1:"Baseline"}
category_contract_duration = pd.api.types.CategoricalDtype(categories=["Single-ride", "1h", "3h", "Baseline"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3", -1:"Baseline"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3","Baseline"], ordered=True)


# Renaming data and applying aliases
a[key_cs] = a[key_cs].map(lambda e:dict_segmentation[e])
a[key_cs] = a[key_cs].astype(category_segmentation)

a[key_cd] = a[key_cd].map(lambda e:dict_contract_duration[e])
a[key_cd] = a[key_cd].astype(category_contract_duration)

a[key_sr] = a[key_sr].map(lambda e:dict_service_rate[e])
a[key_sr] = a[key_sr].astype(category_service_rate)


#a = a.sort_values(by=[key_sr, key_cs, key_cd])
#a = a.sort_values(by=[key_cd, key_cs, key_sr])
a = a.sort_values(by=[key_cs, key_cd, key_sr])



a.rename(columns={'service_rate': 'Service rate',
                    'customer_segmentation': 'Segmentation scenario',
                    'contract_duration': 'Contract duration',
                    'avg_seats': 'Avg',
                    'max_seats': 'Max',
                    'median_seats': 'Median',
                    'distance_cruising': 'Cruising',
                    'distance_rebalancing': 'Rebalancing',
                    'distance_loaded': 'Servicing',
                    'distance_total': 'Total (Km)'}, inplace=True)

#a = a.set_index(['Service rate', 'Segmentation scenario', 'Contract duration'])
#a = a.set_index(['Contract duration', 'Segmentation scenario', 'Service rate'])


a.to_csv("fleet_stats.csv")
a

In [ ]:
d = dict()
d['B+'] = 4544799
d['S+'] = 4580364
d['L+'] = 4560277

a['dif'] = a['Total (Km)']
for k,v in d.items():
    a.loc[a["Segmentation scenario"]==k, "dif"] = a.loc[a["Segmentation scenario"]==k,"dif"].apply(lambda x:x-v)

a = a.set_index(['Segmentation scenario', 'Contract duration', 'Service rate'])
a.to_csv("fleet_stats.csv")


In [ ]:
a

In [ ]:
print(a.to_latex(multicolumn=True, multirow=True))

In [ ]:
df_no_hiring = pd.DataFrame.from_dict(dic_all, orient='index')
df_no_hiring.to_csv("data1.csv")

df_no_hiring

In [ ]:
print(b.to_latex(multicolumn=True, multirow=True))

### Building indexes for multilevel table

In [ ]:
from collections import defaultdict
import math

dict_sl_status = {"MET":"Met", "UNMET":"Unmet"}
filter_sl_status = ["MET"]
instance_attribute = defaultdict(list)
 
# Multilevel service level
for instance_name, dic_request_track in dic_all_request_track.items():
    
    instance_settings = get_instance_settings(instance_name)
    if instance_settings.get('service_rate', '-') == '-':
        continue
    
    column = ('', 'service_rate')
    instance_attribute[column].append(instance_settings['service_rate'])
    column = ('', 'contract_duration')
    instance_attribute[column].append(int(instance_settings['contract_duration']))
    column = ('', 'customer_segmentation')
    instance_attribute[column].append(instance_settings['customer_segmentation'])
    
    for sq_class, sl_status_dic in dic_request_track["service_levels"].items():
        for sl_status, attribute_dic in sl_status_dic.items():
            for attribute, value in attribute_dic.items():
                
                # Filter sl_status
                if sl_status in filter_sl_status:
                    continue
                
                column = (dict_sq_class[sq_class], attribute)
                if(column[1]=="Count"):
                    instance_attribute[column].append(value if not (math.isnan(value) or value==0) else "-")
                else:
                    instance_attribute[column].append(("{:.1f}".format(value/60) if not math.isnan(value) else "-"))


### Table: What is the service level (pickup delay) of the users lying outside SQ-class service rate?

In [ ]:
a = pd.DataFrame.from_dict(instance_attribute)

# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_contract_duration = {3600:"1h", 0:"Single-ride", 10800:"3h"}
category_contract_duration = pd.api.types.CategoricalDtype(categories=["Single-ride", "1h", "3h"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3"], ordered=True)

key_sr = ('', 'service_rate') # (S1, S2, S3)
key_cs = ('', 'customer_segmentation') # (A, AA, BB, etc.)
key_cd = ('', 'contract_duration') #(0, 3600, 18000)

# Filtering data
a = a[a[key_cs].isin(["AA", "BB", "CC"])]

# Renaming data and applying aliases
a[key_cs] = a[key_cs].map(lambda e:dict_segmentation[e])
a[key_cs] = a[key_cs].astype(category_segmentation)

a[key_cd] = a[key_cd].map(lambda e:dict_contract_duration[e])
a[key_cd] = a[key_cd].astype(category_contract_duration)

a[key_sr] = a[key_sr].map(lambda e:dict_service_rate[e])
a[key_sr] = a[key_sr].astype(category_service_rate)

a = a.sort_values(by=[key_sr, key_cs, key_cd])
a = a.set_index([key_sr, key_cs, key_cd])

a.index.names = ["Service rate","Segmentation scenario","Contract duration"]

a

# Getting latex table
# print(a.to_latex(multicolumn=True, multirow=True))